In [ ]:
from selenium import webdriver
import time
import requests
import os
import matplotlib.pyplot as plt
import cv2
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [ ]:
#開啟瀏覽器進入FB
def OpenBrowser():
    profile=webdriver.FirefoxProfile()
    profile.set_preference("dom.webnotifications.enabled",False)
    profile.update_preferences()
    driver=webdriver.Firefox(firefox_profile=profile)
    driver.get("https://www.facebook.com")
    time.sleep(3)
    return driver


In [ ]:
#登入爬蟲帳號
def LoginFB(USERNAME,PASSWORD,driver):
    driver.find_element_by_id("email").send_keys(USERNAME)
    driver.find_element_by_id("pass").send_keys(PASSWORD)
    driver.find_element_by_id("loginbutton").click()
    time.sleep(3)

In [ ]:
#切換至目標頁
def Go_to_Target_Page(target,driver):
    driver.get("https://www.facebook.com/"+target)
    time.sleep(3)

In [ ]:
#取得html資訊
def Get_HTML(driver,rolling):
    for i in range(rolling):
        driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1)))
        time.sleep(2)
    
    htmltext=driver.page_source
    soup=BeautifulSoup(htmltext,'html.parser')
    body=soup.find('body')
    return body

In [ ]:
#取得FB ID
def Get_ID(body):
    FB_ID=body.find('a',{'class':'coverWrap coverImage'})
    DataReferrerid=FB_ID.get('data-referrerid')
    return DataReferrerid

In [ ]:
#取得貼文內容
def Get_Post(body,target):
    post=body.select('div[data-testid="post_message"]')
    post_text=[]
    for i in range(len(post)):
        post_text.append(post[i].text)

    post_list=pd.DataFrame()
    post_list['text']=post_text
    if not os.path.isdir(str(target)):
        os.mkdir(str(target))
    post_list.to_csv(str(target)+'/post_list.csv',index=False)

In [ ]:
#取得封面頁
def Get_CoverBorder(body,target):
    coverBorder=body.find('img',{'class':'coverPhotoImg photo img'})
    cover_URL=coverBorder.get('src')
    #print(cover_URL)

    r=requests.get(cover_URL)
    if not os.path.isdir(target):
        os.mkdir(target)
    with open(str(target)+'/coverBorder.jpg','wb') as f:
        f.write(r.content)

In [ ]:
#取得大頭貼
def Get_BigHead(body,target):
    BigHead=body.find('img',{'class':'_11kf img'})
    BigHead_URL=BigHead.get('src')
    #print(BigHead_URL)

    r=requests.get(BigHead_URL)
    if not os.path.isdir(target):
        os.mkdir(target)
    with open(str(target)+'/BigHead.jpg','wb') as f:
        f.write(r.content)

In [ ]:
#切換至照片頁
def Go_to_Photo_Page(driver,target):
    driver.get("https://www.facebook.com/"+target+"/photos_all")
    time.sleep(3)

In [ ]:
#取得照片
def Get_Photos(driver,target,rolling):
    for i in range(rolling):
        driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1)))
        time.sleep(2)
        
    photo_htmltext=driver.page_source
    photo_soup=BeautifulSoup(photo_htmltext,'html.parser')
    photo_body=photo_soup.find('body')
    photos=photo_body.find_all('li',{'class':'fbPhotoStarGridElement fbPhotoStarGridNonStarred focus_target _53s fbPhotoCurationControlWrapper'})
    print("共有 ",len(photos)," 張圖片")

    photos_URL=[]
    path=str(target)+'/photo'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    fail_count=0
    for i in range(len(photos)):
        try:
            with open(path+'/photo_'+str(i)+'.jpg','wb') as f:
                r=requests.get(photos[i].get('data-starred-src'))
                f.write(r.content)
        except:
            fail_count+=1
    print(len(photos)-fail_count," 張圖片下載成功")
    print(fail_count," 張圖片下載失敗")

In [ ]:
#切換至好友頁
def Go_to_Friend_Page(driver,target):
    driver.get("https://www.facebook.com/"+target+"/friends_all")
    time.sleep(3)

In [ ]:
#取得好友清單 (名字,ID)
def Get_Friends(driver,target,rolling):
    for i in range(rolling):
        driver.execute_script("window.scrollTo(0, {})".format(4000 * (i + 1)))
        time.sleep(2)
    
    friend_htmltext=driver.page_source
    friend_soup=BeautifulSoup(friend_htmltext,'html.parser')
    friend_body=friend_soup.find('body')
    
    friends=friend_body.find_all('li',{'class':'_698'})
    print("共有 ",len(friends)," 位好友")
    friend_name=[]
    friend_ID=[]
    for i in range(len(friends)):
        friend_name.append(friends[i].find('div',{'class':'fsl fwb fcb'}).text)
        try:
            friend_ID.append(friends[i].find('button',{'data-floc':'friends_tab'}).get('data-profileid'))

        except:
            friend_ID.append(friends[i].find('a',{'data-floc':'friends_tab'}).get('data-profileid'))

    friend_list=pd.DataFrame()
    friend_list['name']=friend_name
    friend_list['ID']=friend_ID
    if not os.path.isdir(target):
        os.mkdir(target)
    friend_list.to_csv(str(target)+'/friend_list.csv',index=False)

In [ ]:
driver=OpenBrowser()

In [ ]:
rolling=15
USERNAME="USERNAME"
PASSWORD="PASSWORD"
LoginFB(USERNAME,PASSWORD,driver)

In [ ]:
target='target'#username like "xiao.ming.wang"
Go_to_Target_Page(target,driver)

In [ ]:
body=Get_HTML(driver,rolling)

In [ ]:
ID=Get_ID(body)
print('FB ID: ',ID)

In [ ]:
Get_Post(body,target)

In [ ]:
Get_CoverBorder(body,target)

In [ ]:
Get_BigHead(body,target)

In [ ]:
Go_to_Photo_Page(driver,target)

In [ ]:
Get_Photos(driver,target,rolling)

In [ ]:
Go_to_Friend_Page(driver,target)

In [ ]:
Get_Friends(driver,target,rolling)